In [ ]:
#Acronymes
# electricity price forcasting: epf

In [ ]:
# Project Notes
# Variables
'''
> past electricity price
'''

In [ ]:
# Related Project
# https://github.com/dswigh/green-switch/blob/master/app.py
'''
Web-app to help decide whether to heat their home with gas or electricity
'''
# 

In [ ]:
# Template & Updates

## Motivation

The starting interest is originated from the following youtube [short](https://www.youtube.com/shorts/Sxx-dnNVOdI).

Where billionnaire James Williams caracterised the electricity market with the most Volatile (even beyond cryto) and most reliably predictable market (as it follow laws of physics).

## Goal
How far can a DL model accuratly forcast a day ahead electricity price without any additionnal variable except past price observations?

In [ ]:
# Money Speaking:
'''
How far is profitable (dollar to dollar) to use a DL model for EPF?
'''

## Data

In [ ]:
#sources
#https://transparency.entsoe.eu/
# https://github.com/acruzgarcia/OMIEData
'''
Python package to import data from OMIE 
(Iberian Peninsula's Electricity Market Operator): https://www.omie.es/
'''
# 

## Models

In [ ]:
# Model to test
# > Best models to handle time series forcasting:
'''
    LSTM CNN transformer
    N-BEATS (ElementAI)
    DeepAR (Amazon)
    Spacetimeformer 
    Temporal Fusion Transformer or TFT (Google)
'''
# Training horizon options?
'''

'''

## Web App Solution


In [ ]:
# Template
'''
> Variable Inputs:
 - Capital
 - Financial Cost:
 => Interest Rate (loan cost to invest)
 => Free Risk Rate (bond mostly)
 => Opportunity cost (index fund return?)
 - Investment Horizon
 => forcast the next hour, day, month?
 
> Select Model
 +> Baseline model (ARIMA) for comparaison 

> Results
 - Predicted ROI
 -> Of each model
 - Comparaison with competing investements.
'''

## Reference
[Electricity price forcasting](https://en.wikipedia.org/wiki/Electricity_price_forecasting)
